## Day 76 Lecture 2 Assignment

In this assignment, we will learn about Dask dataframes. We will use the census dataset loaded below and perform some exploratory data analysis.

In [14]:
!python -m pip install "dask[complete]" --quiet
!pip install aiohttp --quiet

In [15]:
import dask.dataframe as dd
import numpy as np

In [16]:
census = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Big%20Data/acs2015_census_tract_data.csv')

In [110]:
census.head(20)

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,0.6,0.0,1503,61838.0,11900.0,25713.0,4548.0,8.1,8.4,34.7,17.0,21.3,11.9,15.2,90.2,4.8,0.0,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,2.3,0.0,1662,32303.0,13538.0,18021.0,2474.0,25.5,40.3,22.3,24.7,21.5,9.4,22.0,86.3,13.1,0.0,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,1.4,0.3,2335,44922.0,5629.0,20689.0,2817.0,12.7,19.7,31.4,24.9,22.1,9.2,12.4,94.8,2.8,0.0,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,0.0,0.0,3306,54329.0,7003.0,24125.0,2870.0,2.1,1.6,27.0,20.8,27.0,8.7,16.4,86.6,9.1,0.0,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,3.8,0.0,7666,51965.0,6935.0,27526.0,2813.0,11.4,17.5,49.6,14.2,18.2,2.1,15.8,88.0,10.5,0.0,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2
5,1001020600,Alabama,Autauga,3851,1787,2064,13.1,72.9,11.9,0.0,0.0,0.0,2642,63092.0,9585.0,30480.0,7550.0,14.4,21.9,24.2,17.5,35.4,7.9,14.9,82.7,6.9,0.0,0.0,6.0,4.5,19.8,1560,79.4,14.7,5.8,0.0,10.9
6,1001020700,Alabama,Autauga,2761,1210,1551,3.8,74.5,19.7,0.0,0.0,0.0,2060,34821.0,7867.0,20442.0,3245.0,28.9,41.9,19.5,29.6,25.3,10.1,15.5,92.4,7.6,0.0,0.0,0.0,0.0,20.0,1166,82.0,14.6,3.4,0.0,11.4
7,1001020801,Alabama,Autauga,3187,1502,1685,1.3,84.0,10.7,3.1,0.0,0.0,2391,73728.0,2447.0,32813.0,4669.0,13.0,25.9,42.8,10.7,34.2,5.5,6.8,84.3,8.1,0.0,0.0,1.7,5.9,24.3,1502,78.1,14.8,7.1,0.0,8.2
8,1001020802,Alabama,Autauga,10915,5486,5429,1.4,89.5,8.4,0.0,0.0,0.0,7778,60063.0,8602.0,24028.0,2233.0,13.9,18.3,31.5,17.5,26.1,7.8,17.1,90.1,8.6,0.0,0.0,0.0,1.3,29.4,4348,73.3,22.1,4.6,0.0,8.7
9,1001020900,Alabama,Autauga,5668,2897,2771,0.4,85.5,12.1,0.0,0.3,0.0,4217,41287.0,7857.0,24710.0,4149.0,6.8,10.0,29.3,13.7,17.7,11.0,28.3,88.7,7.9,0.0,0.0,1.2,2.1,32.9,2485,77.9,15.2,6.9,0.0,7.2


The dataset described above contains the aggregated data for each county in the United States. For example, the number in the Carpool column describes the percent of people in the county who carpool to work.

Print all the column names in the dataset and describe the data in each column

In [40]:
#Answer below:
census.dtypes

CensusTract          int64
State               object
County              object
TotalPop             int64
Men                  int64
Women                int64
Hispanic           float64
White              float64
Black              float64
Native             float64
Asian              float64
Pacific            float64
Citizen              int64
Income             float64
IncomeErr          float64
IncomePerCap       float64
IncomePerCapErr    float64
Poverty            float64
ChildPoverty       float64
Professional       float64
Service            float64
Office             float64
Construction       float64
Production         float64
Drive              float64
Carpool            float64
Transit            float64
Walk               float64
OtherTransp        float64
WorkAtHome         float64
MeanCommute        float64
Employed             int64
PrivateWork        float64
PublicWork         float64
SelfEmployed       float64
FamilyWork         float64
Unemployment       float64
d

How many counties have a unemployement rate above 10%?

In [121]:
#Answer below:
census[['State', 'County']].loc[census.Unemployment > 10.0].drop_duplicates()['County'].count().compute()

2178

Print the number of counties in each state.

In [134]:
# Answer below:
census.groupby(['State', 'County']).count().reset_index()[['State','County']].groupby(['State']).count().reset_index().rename(columns={'County':'County Count'}).compute()

,State,County Count
0,Alabama,67
1,Alaska,29
2,Arizona,15
3,Arkansas,75
4,California,58
5,Colorado,64
6,Connecticut,8
7,Delaware,3
8,District of Columbia,1
9,Florida,67


Add a high child poverty boolean column. This column will be 1 if the rate of child poverty is over %20 and 0 otherwise.

In [155]:
# Answer below:
census['HighChildPoverty'] = census['ChildPoverty'].mask(census['ChildPoverty'] > 20, 1).where(census['ChildPoverty'] > 20, 0)

In [156]:
census.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,HighChildPoverty
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,0.6,0.0,1503,61838.0,11900.0,25713.0,4548.0,8.1,8.4,34.7,17.0,21.3,11.9,15.2,90.2,4.8,0.0,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4,0.0
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,2.3,0.0,1662,32303.0,13538.0,18021.0,2474.0,25.5,40.3,22.3,24.7,21.5,9.4,22.0,86.3,13.1,0.0,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3,1.0
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,1.4,0.3,2335,44922.0,5629.0,20689.0,2817.0,12.7,19.7,31.4,24.9,22.1,9.2,12.4,94.8,2.8,0.0,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2,0.0
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,0.0,0.0,3306,54329.0,7003.0,24125.0,2870.0,2.1,1.6,27.0,20.8,27.0,8.7,16.4,86.6,9.1,0.0,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8,0.0
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,3.8,0.0,7666,51965.0,6935.0,27526.0,2813.0,11.4,17.5,49.6,14.2,18.2,2.1,15.8,88.0,10.5,0.0,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2,0.0


What are the top 10 most populated counties?

In [162]:
#answer below:
census.groupby(['State', 'County']).sum().reset_index()[['State','County', 'TotalPop']].compute().sort_values(by='TotalPop',ascending=False).head(10)

,State,County,TotalPop
204,California,Los Angeles,10038388
610,Illinois,Cook,5236393
2701,Texas,Harris,4356362
103,Arizona,Maricopa,4018143
222,California,San Diego,3223096
215,California,Orange,3116069
362,Florida,Miami-Dade,2639042
1851,New York,Kings,2595259
2657,Texas,Dallas,2485003
1868,New York,Queens,2301139


What is the current number of partitions? 

In [163]:
#Answer below:
census.npartitions


1

#Lecture Notes

In [51]:
chicago = dd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Big%20Data/Current_Employee_Names__Salaries__and_Position_Titles.csv", blocksize=250000)

In [52]:
chicago

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
npartitions=10,,,,,,,,
,object,object,object,object,object,float64,object,object
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [53]:
chicago.compute()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"ALLISON, PAUL W",LIEUTENANT,FIRE,F,Salary,NaN,$107790.00,NaN
1,"BRUNO, KEVIN D",SERGEANT,POLICE,F,Salary,NaN,$104628.00,NaN
2,"COOPER, JOHN E",LIEUTENANT-EMT,FIRE,F,Salary,NaN,$114324.00,NaN
3,"CRESPO, VILMA I",STAFF ASST,LAW,F,Salary,NaN,$76932.00,NaN
4,"DOLAN, ROBERT J",SERGEANT,POLICE,F,Salary,NaN,$111474.00,NaN
...,...,...,...,...,...,...,...,...
283,"ZYGOWICZ, PETER J",POLICE OFFICER,POLICE,F,Salary,NaN,$90024.00,NaN
284,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,$48078.00,NaN
285,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,$87006.00,NaN
286,"ZYRKOWSKI, CARLO E",POLICE OFFICER,POLICE,F,Salary,NaN,$93354.00,NaN


In [54]:
chicago.npartitions

10

In [56]:
chicago_idexed = chicago.set_index('Job Titles')

In [57]:
chicago_idexed.divisions

('1ST DEPUTY INSPECTOR GENERAL',
 'AVIATION SECURITY SERGEANT',
 'DATA ENTRY OPERATOR',
 'FIREFIGHTER (PER ARBITRATORS AWARD)-PARAMEDIC',
 'FOREMAN OF CONSTRUCTION LABORERS',
 'LIEUTENANT-EMT',
 'PLUMBER',
 'POLICE OFFICER',
 'POLICE OFFICER / FLD TRNG OFFICER',
 'SANITATION LABORER',
 'ZONING PLAN EXAMINER')

In [62]:
chicago_salary = chicago[chicago['Salary or Hourly'] == 'Salary']

In [63]:
chicago_salary.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"ALLISON, PAUL W",LIEUTENANT,FIRE,F,Salary,NaN,$107790.00,NaN
1,"BRUNO, KEVIN D",SERGEANT,POLICE,F,Salary,NaN,$104628.00,NaN
2,"COOPER, JOHN E",LIEUTENANT-EMT,FIRE,F,Salary,NaN,$114324.00,NaN
3,"CRESPO, VILMA I",STAFF ASST,LAW,F,Salary,NaN,$76932.00,NaN
4,"DOLAN, ROBERT J",SERGEANT,POLICE,F,Salary,NaN,$111474.00,NaN


In [64]:
chicago_salary['Annual Salary'].head()

0    $107790.00
1    $104628.00
2    $114324.00
3     $76932.00
4    $111474.00
Name: Annual Salary, dtype: object

In [78]:
# chicago_salary['Annual Salary'].map(lambda x: x.strip('$')).astype('float).compute()
chicago_salary['Annual Salary'] = chicago_salary['Annual Salary'].str.strip('$').astype('float')

In [87]:
chicago_salary.groupby('Department').mean()['Annual Salary'].compute()

Department
ADMIN HEARNG             78912.947368
ANIMAL CONTRL            66089.684211
AVIATION                 76140.018777
BOARD OF ELECTION        56051.142857
BOARD OF ETHICS          94552.500000
BUDGET & MGMT            93925.395349
BUILDINGS                98864.833534
BUSINESS AFFAIRS         80446.425000
CITY CLERK               69762.439024
CITY COUNCIL             63577.172069
COMMUNITY DEVELOPMENT    88363.257143
CULTURAL AFFAIRS         87048.909091
DISABILITIES             82431.724138
DoIT                     99681.029703
FAMILY & SUPPORT         79013.588785
FINANCE                  73276.364662
FIRE                     97762.348662
GENERAL SERVICES         83095.528390
HEALTH                   85488.210938
HUMAN RELATIONS          93778.588235
HUMAN RESOURCES          79851.761194
INSPECTOR GEN            84030.666667
IPRA                     94429.285714
LAW                      84582.814404
MAYOR'S OFFICE           96165.512308
OEMC                     73153.778221
P

In [88]:
chicago_salary['Department'].value_counts().compute()

POLICE                   12941
FIRE                      4798
OEMC                       832
PUBLIC LIBRARY             708
AVIATION                   556
FINANCE                    532
HEALTH                     512
TRANSPORTN                 396
WATER MGMNT                366
LAW                        361
CITY COUNCIL               348
FAMILY & SUPPORT           321
STREETS & SAN              321
BUILDINGS                  266
COMMUNITY DEVELOPMENT      210
GENERAL SERVICES           205
BUSINESS AFFAIRS           160
BOARD OF ELECTION          112
DoIT                       101
CITY CLERK                  82
PROCUREMENT                 82
MAYOR'S OFFICE              78
HUMAN RESOURCES             67
CULTURAL AFFAIRS            66
INSPECTOR GEN               63
ANIMAL CONTRL               57
IPRA                        56
BUDGET & MGMT               43
ADMIN HEARNG                38
DISABILITIES                29
TREASURER                   23
COPA                        17
HUMAN RE